# Train Model on Custom Dataset

In [10]:
import torch
import torchvision
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader,random_split
import numpy as np
import math

In [6]:

class winedataset(Dataset):
    def __init__(self):
        ##data loading
        xy = np.loadtxt("/kaggle/input/wine-dataset/wine.csv", delimiter=',', dtype=np.float32, skiprows=1)
        self.x =torch.from_numpy(xy[:, 1:])
        self.y=torch.from_numpy(xy[:,0]).long() - 1   # n_samples, 1 , PyTorch's CrossEntropyLoss expects integer class indices starting from 0).
        print(torch.unique(self.y))
        self.n_samples = xy.shape[0]

        # Normalize the features (zero mean, unit variance)
        self.x = (self.x - self.x.mean(dim=0)) / self.x.std(dim=0)
        
        self.n_samples = xy.shape[0]
        
    def __getitem__(self, index):
        ## dataset index
        return self.x[index], self.y[index]
        
    def __len__(self):
        # len(dataset)
        return self.n_samples
    
dataset  = winedataset()
#first_data = dataset[0]

# Split into training (80%) and testing (20%)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

#features, label = first_data
#print(features, label)

train_loader = DataLoader(dataset=train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=4, shuffle=False)

tensor([0, 1, 2])


In [7]:
dataiter = iter(train_loader)
data = next(dataiter)
features, labels = data
print(features, labels)

tensor([[-0.1855,  0.8357,  0.7782,  0.7501,  0.4382, -1.0308, -1.4309,  1.9135,
         -1.1023,  0.2251, -0.3826, -0.7066, -0.5617],
        [ 0.9477, -0.3906,  1.1427, -0.7171,  1.0683,  1.1263,  0.7616,  0.2262,
          0.1557,  0.5357,  0.7549,  0.4483,  1.9946],
        [ 0.9600,  0.3792, -0.2425,  0.7501, -0.6821, -1.5101, -1.3508,  0.3869,
         -0.9800,  1.9505, -1.1263, -1.3123, -0.4188],
        [ 0.3688, -0.5517, -0.8257, -0.7471, -0.4020,  0.1676,  0.1609, -0.7381,
         -0.4209, -0.4780,  0.2737,  0.2230,  1.7088]]) tensor([2, 0, 2, 0])


In [8]:
epochs =20
total_samples = len(dataset)
n_iteration = math.ceil(total_samples/4)
print(n_iteration)

45


In [11]:
class logisticregression(nn.Module):
    def __init__(self, input_size, output_size):
        super(logisticregression, self).__init__()
        self.linear = nn.Linear(input_size, output_size)
        #self.softmax = nn.Softmax()
    def forward(self, x):
        l = self.linear(x)
        #out = self.softmax(l)
        return l

model = logisticregression(input_size = 13, output_size=3)

In [12]:
loss = nn.CrossEntropyLoss()
learning_rate=0.01
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [13]:
for epoch in range(epochs):
    for i, (inputs, labels) in enumerate(train_loader):

        y_pred = model(inputs)
        #print(y_pred.shape)
        #print(labels.shape)
        
        ls = loss(y_pred,labels )
        
        ls.backward()
        
        optimizer.step()
        optimizer.zero_grad()
    if epoch % 1==0:
            print(f'epoch {epoch}, loss = {ls}')
        

epoch 0, loss = 0.20612242817878723
epoch 1, loss = 0.24632155895233154
epoch 2, loss = 0.28543367981910706
epoch 3, loss = 0.3163605034351349
epoch 4, loss = 0.19303642213344574
epoch 5, loss = 0.043122388422489166
epoch 6, loss = 0.3470191955566406
epoch 7, loss = 0.14330346882343292
epoch 8, loss = 0.17538048326969147
epoch 9, loss = 0.126447394490242
epoch 10, loss = 0.15211397409439087
epoch 11, loss = 0.34275728464126587
epoch 12, loss = 0.0556468665599823
epoch 13, loss = 0.13958750665187836
epoch 14, loss = 0.36210769414901733
epoch 15, loss = 0.11081278324127197
epoch 16, loss = 0.07091362774372101
epoch 17, loss = 0.07719690352678299
epoch 18, loss = 0.06901644170284271
epoch 19, loss = 0.22129756212234497


In [15]:
# Calculate accuracy on the test data
total_correct_test = 0
total_test_samples = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        y_pred_test = torch.argmax(model(inputs), dim=1)  # Get class predictions
        total_correct_test += (y_pred_test == labels).sum().item()
        total_test_samples += labels.size(0)

    test_accuracy = total_correct_test / total_test_samples

print(f"Test Accuracy: {test_accuracy:.4f}")

Test Accuracy: 0.9722
